In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/steam-predict/train_data(7).csv
/kaggle/input/steam-predict/sample_output(15).csv
/kaggle/input/steam-predict/test_data (2).csv


In [2]:
train = pd.read_csv('/kaggle/input/steam-predict/train_data(7).csv')
test = pd.read_csv('/kaggle/input/steam-predict/test_data (2).csv')

In [3]:
train

,AppID,Name,Release date,Estimated owners,Price,Metacritic score,Recommendations,Positive,Negative,Publishers,Genres
0,350110,TransOcean 2: Rivals,"May 10, 2016",100000 - 200000,29.99,69,428,305,281,astragon Entertainment,"Simulation,Strategy"
1,312780,Way of the Samurai 4,"July 23, 2015",200000 - 500000,24.99,72,1475,1288,318,"Spike Chunsoft Co., Ltd.","Action,Adventure"
2,230150,Incredipede,"March 18, 2013",50000 - 100000,9.99,74,140,243,96,Northway Games,"Action,Adventure,Casual,Indie,Simulation"
3,1252300,RetroMania Wrestling,"February 25, 2021",0 - 20000,29.99,66,174,211,30,Retrosoft Studios,"Action,Sports"
4,760650,Hammerting,"November 16, 2021",100000 - 200000,9.99,64,1358,1328,705,Team17 Digital,"Adventure,Indie,RPG,Simulation,Strategy"
...,...,...,...,...,...,...,...,...,...,...,...
2995,266410,iRacing,"January 12, 2015",100000 - 200000,9.99,79,1281,2096,334,"iRacing.com Motorsport Simulations,iRacing","Massively Multiplayer,Racing,Simulation,Sports"
2996,939100,Darksburg,"September 23, 2020",100000 - 200000,14.99,61,1489,1511,750,Shiro Unlimited,"Action,Indie"
2997,1931770,Chants of Sennaar,"September 05, 2023",100000 - 200000,19.99,85,6125,6484,82,Focus Entertainment,"Adventure,Indie"
2998,41500,Torchlight,"October 27, 2009",1000000 - 2000000,14.99,83,4323,5424,495,Runic Games,RPG


In [4]:
categorical = ['Name', 'Publishers', ]

In [5]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

def estimate(estimated_val):
    low, high = estimated_val.split(' - ')
    return (int(low) + int(high)) / 2

def preprocess(df, fit=False):
    df = df.copy()
    df.bfill(inplace=True)

    df['Avg owners'] = df['Estimated owners'].apply(estimate)
    df.drop(columns = ['Estimated owners'], inplace = True)

    df['Release date'] = pd.to_datetime(df['Release date'])
    df['day'] = df['Release date'].dt.day
    df['month'] = df['Release date'].dt.month
    df['year'] = df['Release date'].dt.year
    df['dayofweek'] = df['Release date'].dt.dayofweek
    df.drop(columns=['Release date'], inplace=True)

    df = df.drop(columns=['Name', 'Publishers'])

    df['Genres'] = df['Genres'].fillna('')
    df['Genres'] = df['Genres'].apply(lambda x: [g.strip() for g in x.split(',')])

    if fit:
        genres_encoded = mlb.fit_transform(df['Genres'])
    else:
        genres_encoded = mlb.transform(df['Genres'])

    genres_df = pd.DataFrame(genres_encoded, 
                             columns=[f"Genre_{g}" for g in mlb.classes_],
                             index=df.index)

    df = df.drop(columns=['Genres'])
    df = pd.concat([df, genres_df], axis=1)

    return df

In [6]:
train = preprocess(train, fit = True)

In [7]:
import xgboost as xgb

xgb = xgb.XGBRegressor(n_estimators=2000,
                       learning_rate=0.1,
                       max_depth=6,
                       subsample=0.8,
                       colsample_bytree=0.8,
                       random_state=42,
                       n_jobs=-1,
                       verbosity=2)

In [8]:
import re
x_train = train.drop(columns = [ 'AppID','Price'])
y_train = train['Price']

x_train.columns = [
    re.sub(r'[\[\]<]', '_', str(c)) for c in x_train.columns
]

In [9]:
x_train

,Metacritic score,Recommendations,Positive,Negative,Avg owners,day,month,year,dayofweek,Genre_Action,...,Genre_Indie,Genre_Massively Multiplayer,Genre_Nudity,Genre_RPG,Genre_Racing,Genre_Sexual Content,Genre_Simulation,Genre_Sports,Genre_Strategy,Genre_Violent
0,69,428,305,281,150000.0,10,5,2016,1,0,...,0,0,0,0,0,0,1,0,1,0
1,72,1475,1288,318,350000.0,23,7,2015,3,1,...,0,0,0,0,0,0,0,0,0,0
2,74,140,243,96,75000.0,18,3,2013,0,1,...,1,0,0,0,0,0,1,0,0,0
3,66,174,211,30,10000.0,25,2,2021,3,1,...,0,0,0,0,0,0,0,1,0,0
4,64,1358,1328,705,150000.0,16,11,2021,1,0,...,1,0,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,79,1281,2096,334,150000.0,12,1,2015,0,0,...,0,1,0,0,1,0,1,1,0,0
2996,61,1489,1511,750,150000.0,23,9,2020,2,1,...,1,0,0,0,0,0,0,0,0,0
2997,85,6125,6484,82,150000.0,5,9,2023,1,0,...,1,0,0,0,0,0,0,0,0,0
2998,83,4323,5424,495,1500000.0,27,10,2009,1,0,...,0,0,0,1,0,0,0,0,0,0


In [10]:
xgb.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=2000, n_jobs=-1,
             num_parallel_tree=None, random_state=42, ...)

In [11]:
test = preprocess(test)

In [12]:
x_test = test.drop(columns = ['AppID'])
x_test.columns = [
    re.sub(r'[\[\]<]', '_', str(c)) for c in x_train.columns
]

In [13]:
preds = xgb.predict(x_test)

In [14]:
submission = pd.DataFrame({
    'subtaskID' : (['1'] * len(x_test)) + (['2'] * len(x_test)),
    'datapointID' : test['AppID'].tolist() + test['AppID'].tolist(),
    'answer' : test['Avg owners'].tolist() + preds.tolist()
})

In [15]:
submission.to_csv('submission.csv', index = False)